In [1]:
from pathlib import Path
thispath = Path('__file__').resolve()
base_path = thispath.parent.parent
import sys; sys.path.insert(0, str(base_path))
import json
sys.path.append('../utils/')
import time

In [2]:
import SimpleITK as sitk
import pandas as pd
from tqdm.auto import tqdm
import time
import numpy as np
from metrics import get_landmarks_array_from_txt_file, target_registration_error
from elastic_helpers import elastix_wrapper, modify_field_parameter_map,transformix_wrapper

In [3]:
base_path = Path().resolve().parent
params_path = base_path / 'parameters'
output_path = base_path /'elastix'
results_path = base_path / 'experiments' / 'elastik' / 'results.json'


In [4]:
dataset = pd.read_csv('../result_data/dir_lab_copd.csv')

In [5]:
def registration(dataset,param_maps_to_use,experiment_name,masks=False):
    results = {}

    for i in tqdm(range(len(dataset))):
        sample = dataset.iloc[i]
        sample_data = sample['case']
        sample_mean = sample['disp_mean']
        sample_std = sample['disp_std']

        pm = param_maps_to_use[-1]
        data_path = base_path / 'result_data' / sample_data
        output_pm_path = output_path / pm.rstrip('.txt') / experiment_name
        output_pm_path.mkdir(exist_ok=True, parents=True)

        parameters_filename = params_path
        result_path = output_path / sample['case'] / experiment_name
        result_path.mkdir(exist_ok=True, parents=True)
        
        param_maps_to_use_ = [str(params_path / p) for p in param_maps_to_use]
        name = sample['case']
        i_temp_path = data_path / f'{sample_data}_iBHCT.nii.gz'
        e_temp_path = data_path / f'{sample_data}_eBHCT.nii.gz'
        i_lungs_mask_temp_path = data_path / f'{sample_data}_iBHCT_masks.nii.gz'
        e_lungs_mask_temp_path = data_path / f'{sample_data}_eBHCT_masks.nii.gz'
        
        print('Registration of image...')
        start = time.time()
        
        if mask is False:
            transform_map_path = elastix_wrapper(
                i_temp_path, e_temp_path, res_path.parent, param_maps_to_use_,
                verbose=False, keep_just_useful_files=False
            )
        else:
            transform_map_path = elastix_wrapper(
                i_temp_path, e_temp_path, res_path.parent, param_maps_to_use_,
                i_lungs_mask_temp_path, e_lungs_mask_temp_path, verbose=False,
                keep_just_useful_files=False
            )
        
        
        reg_time = time.time() - start
        name = f'{sample_data}_300_iBH_xyz_r1.txt'
        lm_points_filepath = data_path / name
        
        field_value_pairs = [
            ('ResultImageFormat', 'nii.gz'),
            ('ResultImagePixelType', "float"),
            ('FinalBSplineInterpolationorder', '0')
        ]

        modify_field_parameter_map(field_value_pairs, transform_map_path)
        
        # Transform landmarks
        print('Transforming points...')
        lm_out_filepath = result_path.parent / f'r_{sample_data}'
        transformix_wrapper(
            lm_points_filepath, lm_out_filepath, transform_map_path,
            points=True)
                
        landmarks = get_landmarks_array_from_txt_file(lm_out_filepath / name)
        e_landmark_pts = pd.read_csv(sample['e_landmark_pts'], header=None, index_col=None).values
        m, s = target_registration_error(landmarks, e_landmark_pts, [sample['space_x'],sample['space_y'],sample['space_z']])
        print(f'{"-"*10} {sample_data} {"-"*10}')
        print(f'Provided displacement: {sample_mean} | {sample_std} ')
        print(f'Computed displacement: {m} | {s}')
        
        results[sample['case']] = {
            'mean_tre': m,
            'std_tre': s,
            'time': reg_time,
            'maks_on' : masks
        }
    
    return results

## Experiments without a mask

In [6]:
experiment_results = {}
experiment_name = 'Baseline'
param_maps_to_use = [
        'Par0054_sstvd.txt'
]

In [7]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 21.03 | 10.12
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 18.76 | 6.0
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 7.59 | 4.84
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 24.55 | 9.36


In [8]:
experiment_name = 'Par0008_affine'
param_maps_to_use = [
        'Parameters.Par0008.affine.txt'
]

In [9]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 25.9 | 10.99
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 23.86 | 5.18
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 6.04 | 3.49
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 21.61 | 9.59


In [10]:
experiment_name = 'Par0003_affine'
param_maps_to_use = [
        'Par0003.affine.txt'
]

In [11]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 25.91 | 10.99
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 24.61 | 5.38
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.9 | 3.33
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 21.62 | 9.58


In [14]:
experiment_name = 'MI.Coarse.Bspline_tuned'
param_maps_to_use = [
        'MI.Coarse.Bspline_tuned.txt'
]

In [15]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 8.8 | 4.81
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 13.15 | 6.29
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 4.91 | 2.97
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 9.86 | 4.04


In [16]:
experiment_name = 'MI.Fine.Bspline_tuned'
param_maps_to_use = [
        'MI.Fine.Bspline_tuned.txt'
]

In [17]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 26.33 | 11.42
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 21.79 | 6.46
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 12.64 | 6.38
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 29.58 | 12.92


In [18]:
experiment_name = 'MI.RP.Bspline_tuned'
param_maps_to_use = [
        'MI.RP.Bspline_tuned.txt'
]

In [19]:
experiment_results[experiment_name] = \
    registration(dataset,param_maps_to_use,experiment_name)
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))

  0%|          | 0/4 [00:00<?, ?it/s]

Registration of image...
Transforming points...
---------- copd1 ----------
Provided displacement: 25.9 | 11.57 
Computed displacement: 8.84 | 5.33
Registration of image...
Transforming points...
---------- copd2 ----------
Provided displacement: 21.77 | 6.46 
Computed displacement: 16.06 | 7.61
Registration of image...
Transforming points...
---------- copd3 ----------
Provided displacement: 12.29 | 6.39 
Computed displacement: 5.3 | 2.95
Registration of image...
Transforming points...
---------- copd4 ----------
Provided displacement: 30.9 | 13.49 
Computed displacement: 9.66 | 3.74


## Adding GT

In [21]:
experiment_results['GT'] = {'copd1': {'mean_tre': 25.90, 'std_tre': 11.57, 'time': 0}, 
                            'copd2': {'mean_tre': 21.77, 'std_tre': 6.46 , 'time': 0}, 
                            'copd3': {'mean_tre': 12.29, 'std_tre': 6.39, 'time': 0}, 
                            'copd4': {'mean_tre': 30.90, 'std_tre': 13.49, 'time': 0}}

In [22]:
with open(results_path, 'w') as json_file:
    json.dump(experiment_results, json_file, indent=4, separators=(',', ': '))